# Spectrogram data preparation

Aim: create a pipeline which creates a dataset of (spectrogram, caption) pairs

1. Load from huggingface
2. Inspect data
3. Prepare dataset of first speaker


In [2]:
import torchaudio
import librosa
import librosa.display
import numpy as np
from torchaudio.transforms import Spectrogram
from torchvision.transforms import functional as tf
import matplotlib.pyplot as plt

In [3]:
# Check available datasetes

from datasets import load_dataset, list_datasets

""" dataset_lists = list_datasets()

print(', '.join(dataset for dataset in dataset_lists)) """

" dataset_lists = list_datasets()\n\nprint(', '.join(dataset for dataset in dataset_lists)) "

In [35]:
dataset = load_dataset('lj_speech', split='train')

Generating train split:   0%|          | 0/13100 [00:00<?, ? examples/s]

Dataset lj_speech downloaded and prepared to /home/ryan/.cache/huggingface/datasets/lj_speech/main/1.1.0/6e03b5308ab68705d902e2bbeaff59feb283307770fafb8d6704e604b6d179aa. Subsequent calls will reuse this data.


In [36]:
print(dataset)
dataset[0]

Dataset({
    features: ['id', 'audio', 'file', 'text', 'normalized_text'],
    num_rows: 13100
})


{'id': 'LJ001-0001',
 'audio': {'path': '/home/ryan/.cache/huggingface/datasets/downloads/extracted/767c76e13c24c1cc26025e0fbcf1d4fe53cfaa6521aac12a67eac69b9fa07ab2/LJSpeech-1.1/wavs/LJ001-0001.wav',
  'array': array([-7.32421875e-04, -7.62939453e-04, -6.40869141e-04, ...,
          7.32421875e-04,  2.13623047e-04,  6.10351562e-05]),
  'sampling_rate': 22050},
 'file': '/home/ryan/.cache/huggingface/datasets/downloads/extracted/767c76e13c24c1cc26025e0fbcf1d4fe53cfaa6521aac12a67eac69b9fa07ab2/LJSpeech-1.1/wavs/LJ001-0001.wav',
 'text': 'Printing, in the only sense with which we are at present concerned, differs from most if not from all the arts and crafts represented in the Exhibition',
 'normalized_text': 'Printing, in the only sense with which we are at present concerned, differs from most if not from all the arts and crafts represented in the Exhibition'}

Mel spectrogram conversion

In [37]:
import torchaudio
import torchaudio.transforms as transforms

# Takes an existing dataset and creates a copy with a spectrogram columm - no prefix
def transform_wav(dataset):    
    transformed_data = []

    for data in dataset:
        waveform, sample_rate = torchaudio.load(data['audio']['path'], normalize=True)

        # Parameters: number of fourier bins & mel filter banks
        n_fft = 256 
        n_mels = 80

        transform = transforms.MelSpectrogram(
            sample_rate=sample_rate,
            n_fft=n_fft,
            n_mels=n_mels
        )
        
        spectrogram = transform(waveform)
        
        data['spectrogram'] = spectrogram
        transformed_data.append(data)
        
    return transformed_data

# Takes an existing dataset and creates a copy with a spectrogram column, allows selection of prefix
def transform_wav(dataset, prefix):
    prefix_len = len(prefix)
    filtered_data = [d for d in dataset if d["id"][:prefix_len] == prefix]
    
    transformed_data = []

    for data in filtered_data:
        waveform, sample_rate = torchaudio.load(data['audio']['path'], normalize=True)

        # Parameters: number of fourier bins & mel filter banks
        n_fft = 256 
        n_mels = 80

        transform = transforms.MelSpectrogram(
            sample_rate=sample_rate,
            n_fft=n_fft,
            n_mels=n_mels
        )
        
        spectrogram = transform(waveform)
        
        data['spectrogram'] = spectrogram
        transformed_data.append(data)
        
    return transformed_data
        
""" 
Transformation back into
invers_transform = torchaudio.transforms.InverseMelScale(sample_rate=sample_rate, n_stft=n_stft)
grifflim_transform = torchaudio.transforms.GriffinLim(n_fft=n_fft)

mel_specgram = transform(waveform)
inverse_waveform = invers_transform(mel_specgram)
pseudo_waveform = grifflim_transform(inverse_waveform) """
        
test_dataset = transform_wav(dataset, "LJ001")
    

/home/ryan/miniconda3/envs/msc_diss/lib/python3.10/site-packages/torchaudio/functional/functional.py:576: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (80) may be set too high. Or, the value for `n_freqs` (129) may be set too low.
  warnings.warn(


{'id': 'LJ001-0001',
 'audio': {'path': '/home/ryan/.cache/huggingface/datasets/downloads/extracted/767c76e13c24c1cc26025e0fbcf1d4fe53cfaa6521aac12a67eac69b9fa07ab2/LJSpeech-1.1/wavs/LJ001-0001.wav',
  'array': array([-7.32421875e-04, -7.62939453e-04, -6.40869141e-04, ...,
          7.32421875e-04,  2.13623047e-04,  6.10351562e-05]),
  'sampling_rate': 22050},
 'file': '/home/ryan/.cache/huggingface/datasets/downloads/extracted/767c76e13c24c1cc26025e0fbcf1d4fe53cfaa6521aac12a67eac69b9fa07ab2/LJSpeech-1.1/wavs/LJ001-0001.wav',
 'text': 'Printing, in the only sense with which we are at present concerned, differs from most if not from all the arts and crafts represented in the Exhibition',
 'normalized_text': 'Printing, in the only sense with which we are at present concerned, differs from most if not from all the arts and crafts represented in the Exhibition',
 'spectrogram': tensor([[[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [0.